In [55]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from classification.models.crnn.model import AudioCRNN # this might change depending on import mechanism

import torch
from datasets.audioset import Audioset
from torchvision import transforms
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## CRNN

Zero pads input to max length in batch. Required for Pytorch RNN's.

Note that `fixed_padding=False` is given to Audioset instance and `collate_fn=audio_set_val.pad_seq` is given to Dataloader.

In [ ]:
# audio_set_tr = Audioset(split_mode='training', transforms=dataset_transforms)
# audio_set_val = Audioset(split_mode='validation', transforms=dataset_transforms)
audio_set_val = Audioset(split_mode='validation', fixed_padding=False)
# tr_loader = DataLoader(audio_set_tr, batch_size=32)

In [ ]:
val_loader = DataLoader(audio_set_val, batch_size=2, collate_fn=audio_set_val.pad_seq)

In [ ]:
res_path = '/nfs/students/summer-term-2020/project-4/data/models/crnn.pth'
checkpoint = torch.load(res_path)

model = AudioCRNN(checkpoint['classes'], checkpoint['config'])
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device) 
model.eval()

In [ ]:
for i, data in enumerate(val_loader):
    data = {k:v.to(device) for k,v in data.items()}
    batch = {
        'audio': data['audio'],
        'lengths': data['lengths']
    }
    print(model(batch))
    break

## RecursiveCNN

Zero pads input to max length of dataset.

Note that `fixed_padding=True` is given to Audioset instance.

In [ ]:
audio_set_val = Audioset(split_mode='validation', fixed_padding=True)
val_loader = DataLoader(audio_set_val, batch_size=1)

In [ ]:
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN

model_state_dict_path = "/nfs/students/summer-term-2020/project-4/data/models/DeepRecursiveCNN.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model = model.eval()
model = model.to(device) 

In [ ]:
for i, data in enumerate(val_loader):
    data = {k:v.to(device) for k,v in data.items()}
    batch = {
        'audio': data['audio']
    }
    print(model(batch))
    break